In [1]:
import keras
import pickle
import gensim
import csv
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout
from keras.models import Model

Using Theano backend.


In [2]:
def transform_sent(sent):
    '''
    Takes as input a tokenized sentence (list of strings) and outputs a numpy array
    where each row is the word vector for the corresponding word in the sentence.
    Final matrix has dimensionality of len(sent) x dimensionality of word vectors
    '''
    vector = []
    for w in sent:
        if w == '<PAD>' or w not in model.vocab:
            vector.append(np.ones((300,)))
        else:
            vector.append(model[w])
    vector = np.array(vector)
    
    return vector

def pad_sentences(text):
    maxlen = max([len(sent) for sent in text])
    for sent in text:
        while len(sent) < maxlen:
            sent.append('<PAD>')
            
    return text

In [3]:
tokenized = pickle.load(open('../pickles/sarcasm/sarcasm_tokenized.pkl', 'rb'))
model = gensim.models.Word2Vec.load_word2vec_format('../GoogleNews-vectors-negative300.bin.gz', binary=True)

In [4]:
datafile = "../data/sarcasm_v2.csv"

def load_data():
    with open(datafile) as f:
        return list(csv.reader(f))

data = load_data()
features = [[transform_sent(sent) for sent in doc] for doc in tokenized]
labels = [line[1] for line in data if line[0]=="GEN"]

In [27]:
X = np.array([transform_sent(s).flatten() for s in new_tokenized])

In [5]:
new_tokenized = pad_sentences(tokenized)

In [28]:
x_train = X[:2860]
x_test = X[2860:]

In [18]:
encoding_dim = 64

In [23]:
input_sequence = Input(shape=(198*300,))

encoded = Dense(encoding_dim, activation='relu')(input_sequence)

decoded = Dense(198*300, activation='sigmoid')(encoded)

autoencoder = Model(input=input_sequence, output=decoded)
encoder = Model(input=input_sequence, output=encoded)
encoder.load_weights('encoded')

In [24]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

In [196]:
input_sequence = Input(shape=(198,300))

encoded = LSTM(64)(input_sequence)

decoded = RepeatVector(198)(encoded)
decoded = LSTM(300, return_sequences=True)(decoded)

sequence_autoencoder = Model(input_sequence, decoded)
encoder = Model(input_sequence, encoded)

In [25]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [29]:
autoencoder.fit(x_train, x_train,
                nb_epoch=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 2860 samples, validate on 400 samples
Epoch 1/50
2860/2860 [==============================] - 14s - loss: 0.6924 - val_loss: 0.6880
Epoch 2/50
2860/2860 [==============================] - 14s - loss: 0.6354 - val_loss: 0.4738
Epoch 3/50
2860/2860 [==============================] - 14s - loss: 0.3267 - val_loss: 0.3190
Epoch 4/50
2860/2860 [==============================] - 13s - loss: 0.2633 - val_loss: 0.3084
Epoch 5/50
2860/2860 [==============================] - 12s - loss: 0.2568 - val_loss: 0.3034
Epoch 6/50
2860/2860 [==============================] - 12s - loss: 0.2533 - val_loss: 0.2984
Epoch 7/50
2860/2860 [==============================] - 12s - loss: 0.2505 - val_loss: 0.2981
Epoch 8/50
2860/2860 [==============================] - 13s - loss: 0.2479 - val_loss: 0.2925
Epoch 9/50
2860/2860 [==============================] - 13s - loss: 0.2456 - val_loss: 0.2910
Epoch 10/50
2860/2860 [==============================] - 15s - loss: 0.2432 - val_loss: 0.2911
Epoch 11/50


KeyboardInterrupt: 

In [202]:
encoded_sents = encoder.predict(X)

In [203]:
pickle.dump(encoded_sents, open('LSTM_encoded', 'wb'))

In [204]:
encoder.save_weights('LSTM_encoder_weights')

In [11]:
autoencoder.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 59400)         0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 15000)         891015000   input_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1000)          15001000    dense_1[0][0]                    
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 300)           300300      dense_2[0][0]                    
___________________________________________________________________________________________